###Install robustness

In [1]:
!pip install robustness

     |████████████████████████████████| 81kB 2.2MB/s 
     |████████████████████████████████| 460kB 8.3MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 204kB 35.2MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=415115e03d568afd17a87918b74b47b5c4636f48897655ce795ea5f04cb2e8a8
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil


###Imports

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from robustness.model_utils import make_and_restore_model
from robustness.datasets import CIFAR
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

###Load and prepare CIFAR-10

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
ds = CIFAR('data') 

In [0]:
device = torch.device("cuda:0")

###Load model

In [0]:
model, _ = make_and_restore_model(arch='resnet50', dataset=ds, resume_path='cifar_l2_0_25.pt')
model.eval()
model.to(device)

In [0]:
_, test_loader = ds.make_loaders(workers=0, batch_size=1, shuffle_train=False, shuffle_val=False)
_, test_loader_shuffled = ds.make_loaders(workers=0, batch_size=1, shuffle_train=True, shuffle_val=True)

In [0]:
def compute_accuracy(logits, y_true, device='cuda:0'):
    y_pred = torch.argmax(logits, dim=1)
    y_true_on_device = y_true.to(device)
    accuracy = (y_pred == y_true_on_device).float().mean()
    return accuracy

In [0]:
criterion = nn.MSELoss()

In [0]:
lr = 0.01

###Generation of robust dataset

In [0]:
#генерация робустного датасета
robust_ds = []

for i in tqdm(range(len(test_loader))):
    X_batch, y_batch = next(iter(test_loader))
    X_rand_batch, y_rand_batch = next(iter(test_loader_shuffled))
    X_batch_gpu = X_batch.to(device)
    X_rand_batch_gpu = X_rand_batch.to(device)

    n = 0
    while n < 1000:
        X_rand_batch_gpu = np.clip(X_rand_batch_gpu.detach().cpu(), 0, 1)
        X_rand_batch_gpu.requires_grad_(True)
        logits = model(X_batch_gpu)[0]
        logits_rand = model(X_rand_batch_gpu)[0]
        loss = criterion(logits, logits_rand)
        grad = torch.autograd.grad(loss, X_rand_batch_gpu)
        x_clone = X_rand_batch_gpu.clone()
        x_clone -= lr*grad[0]
        X_rand_batch_gpu = x_clone.clone()
        n = n + 1
    robust_ds.append(X_rand_batch_gpu)


In [0]:
torch.save(robust_ds, 'robust_dataset.pt')

###Explore robust dataset and compare it with original CIFAR-10

In [0]:
n_pics_to_show = 10
fig, ax = plt.subplots(2, n_pics_to_show, figsize=(20, 10))

for i in range(n_pics_to_show):
    rand_idx = np.random.randint(len(robust_ds))

    pic = robust_ds[rand_idx][0]
    pic2 = testset[rand_idx][0]

    pic_np = pic.cpu().data.numpy()

    pic_np = np.rollaxis(pic_np, 0, 3) # 3xHxW to HxWx3

    pic_np2 = pic2.cpu().data.numpy()
    pic_np2 = np.rollaxis(pic_np2, 0, 3) # 3xHxW to HxWx3

    ax[0, i].imshow(pic_np)


    ax[1, i].imshow(pic_np2)

